In [65]:
# Load packages

import numpy as np 
import pandas as pd 

import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objects as go

import json
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, Polygon, MultiPoint

import re


# Join scraped AWC details with AWC geo-location. Some of the values for lat and lon are incorrect or zeroes in the scraped data.

df_awc_kar = pd.read_csv('awc_karnataka_districts.csv')

df_572 = pd.read_csv('awc_monitoring_details_final_572.csv') # Bangalore Urban
df_580 = pd.read_csv('awc_monitoring_details_final_580.csv') # Yadgir
df_569 = pd.read_csv('awc_monitoring_details_final_569.csv') # Udupi
df_561 = pd.read_csv('awc_monitoring_details_final_561.csv') # Gadag
df_560 = pd.read_csv('awc_monitoring_details_final_560.csv') # Koppal

df_scraped = pd.concat([df_572, df_580, df_569, df_561, df_560], ignore_index=True, sort=False)

df_scraped['awc_code'] = df_scraped['awc_code'].astype('str')
df_scraped['awc_code'] = df_scraped['awc_code'].str.strip()
df_awc_kar['awc_code'] = df_awc_kar['awc_code'].str.strip()
df_awc_kar['awc_code'] = df_awc_kar['awc_code'].apply(lambda x: re.sub('[^0-9]','', x) )

df_merged = df_awc_kar.merge(df_scraped, left_on ='awc_code', right_on = 'awc_code', how='right')

df_merged['snp_3_5'] = df_merged['NoBoys3_5SNP'] + df_merged['NoGirls3_5SNP']

df_merged['mod_mal'] = df_merged['BoysMod0_3NS'] + df_merged['GirlsMod0_3NS']
+ df_merged['BoysMod3_5NS'] + df_merged['GirlsMod3_5NS']

df_merged['sev_mal'] = df_merged['BoysSev0_3NS'] + df_merged['GirlsSev0_3NS']
+ df_merged['BoysSev3_5NS'] + df_merged['GirlsSev3_5NS']

df_merged['SNB_NOBoys_SC_36m_72m'] = df_merged['SNB_NOBoys_SC_36m_72m'].fillna(0).astype('int')


df_merged = df_merged.fillna('')

#df_merged_no_water = df_merged[df_merged['DrnkWater'] == 'No']
#df_merged_no_toilet = df_merged[df_merged['Toilet'] == 'No']

# Plot AWC
text_on_hover=['AWC:' + df_merged['awc_name'][i] 
                  + '<br>' + 'Contact:' + df_merged['AwwNameCont'][i]
                  + '<br>' + 'Code:' + df_merged['awc_code'][i] + ' Alpha:' + df_merged['awc_alpha_code'][i]
                  + '<br>' + 'Addr:' + df_merged['NousePin'][i]
                  + '<br>' + 'SNP_3_5:' + str(df_merged['snp_3_5'][i])
                  + '<br>' + 'SNB_NOBoys_SC_36m_72m:' + str(df_merged['SNB_NOBoys_SC_36m_72m'][i])               
                  + '<br>' + 'SNP_Women:' + str(df_merged['WomenSNP'][i])
                  + '<br>' + 'Mod_Mal:' + str(df_merged['mod_mal'][i])
                  + '<br>' + 'Sev_Mal:' + str(df_merged['sev_mal'][i])
                  + '<br>' + 'Type:' + df_merged['typeAwc'][i] + str(', ')+ df_merged['AWC'][i] + str(', ')
                                     + df_merged['OwnBuild'][i] + str(', ')
                                     + df_merged['PuccaBuild'][i] 
                  + '<br>' + 'Water:' + df_merged['DrnkWater'][i] + str(' Toilet:')+ df_merged['Toilet'][i] 
                  
                  
                  for i in range(df_merged.shape[0])]

fig = go.FigureWidget(
    data=
    
    [
        go.Scattermapbox(
            lat=df_merged['lat'],
            lon=df_merged['lon'],
            mode="markers",
            text=text_on_hover,
            hoverinfo='all',
            name='AWC location',
            marker=go.scattermapbox.Marker(symbol='circle', size=4, color='blue', opacity=0.6)
        )
    ]
    +
    [
        go.Scattermapbox(
            lat=df_merged['lat'],
            lon=df_merged['lon'],
            mode="markers",
            text=text_on_hover,
            hoverinfo='all',
            name='SNP 3 to 5 yrs count',
            visible='legendonly',
            marker=go.scattermapbox.Marker(symbol='circle', size=4, color=df_merged['snp_3_5'], colorscale="viridis_r", opacity=0.8)
        )
    ]
    +
    [
        go.Scattermapbox(
            lat=df_merged['lat'],
            lon=df_merged['lon'],
            mode="markers",
            text=text_on_hover,
            hoverinfo='all',
            name='SNP 3 to 5 yrs count 90th percentile',
            visible='legendonly',
            marker=go.scattermapbox.Marker(symbol='circle', size=4, color=(df_merged['snp_3_5'] > df_merged[df_merged['snp_3_5'] > 0]['snp_3_5'].quantile(0.9)).astype('int'), colorscale="viridis_r", opacity=0.8)

        )
    ]
    +
    [
        go.Scattermapbox(
            lat=df_merged['lat'],
            lon=df_merged['lon'],
            mode="markers",
            text=text_on_hover,
            hoverinfo='all',
            name='SNP SC Boys 3 to 6 yrs count',
            visible='legendonly',
            marker=go.scattermapbox.Marker(symbol='circle', size=4, color=df_merged['SNB_NOBoys_SC_36m_72m'], colorscale="viridis_r", opacity=0.8)
        )
    ]

    +
    [
        go.Scattermapbox(
            lat=df_merged['lat'],
            lon=df_merged['lon'],
            mode="markers",
            text=text_on_hover,
            hoverinfo='all',
            name='Moderately Malnourished 0 to 5 yrs',
            visible='legendonly',
            marker=go.scattermapbox.Marker(symbol='circle', size=4, color=df_merged['mod_mal'], colorscale="viridis_r", opacity=0.8)
        )
    ]
    +
    [
        go.Scattermapbox(
            lat=df_merged['lat'],
            lon=df_merged['lon'],
            mode="markers",
            text=text_on_hover,
            hoverinfo='all',
            name='Severly Malnourished 0 to 5 yrs',
            visible='legendonly',
            marker=go.scattermapbox.Marker(symbol='circle', size=4, color=df_merged['sev_mal'], colorscale="viridis_r", opacity=0.8)
        )
    ]
  
    +
    [
        go.Scattermapbox(
            lat=df_merged['lat'],
            lon=df_merged['lon'],
            mode="markers",
            text=text_on_hover,
            hoverinfo='all',
            name='SNP women count',
            visible='legendonly',
            marker=go.scattermapbox.Marker(symbol='circle', size=4, color=df_merged['WomenSNP'], colorscale="viridis_r", opacity=0.8)
        )
    ]


)

fig.update_layout(
#                  mapbox_style="open-street-map",
                  mapbox_style="carto-positron",
#                  mapbox_style="white-bg",
#                  mapbox_style="stamen-terrain",
                  mapbox_zoom=6, 
                  mapbox_center = {"lat": 15.31, "lon": 75.71},
                  margin={"r":0,"t":0,"l":0,"b":0},
                  title={
                    'text': 'Aanganwadi Centres', 
                    'y':0.97,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'})

fig.update_layout(
    legend=dict(
        x=1,
        y=.85,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        itemsizing="trace",
        bordercolor="peru",
        borderwidth=2
    )
)


#create html file
fig.write_html('awc_scatter_karnataka.html', auto_open=True)

#====== code ends here ======